In [91]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [70]:
# Load the training and test datasets
train_data_path = '/content/CAH-201803-train.csv'
test_data_path = '/content/CAH-201803-test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [71]:
train_data

,id_num,Q1,Q2,political_affiliation,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18
0,1,Male,53,Independent,Liberal,College degree,Black,No,No,No,"Yes, somewhat religious",Pro-Choice,No,No,Behave no differently,5,2,5,No
1,5,Female,66,Independent,Conservative,Some college,White,Yes,No,Yes,"Yes, very religious",Pro-life,Yes,Yes,Less Willing,4,5,4,No
2,7,Female,58,Democrat,Liberal,College degree,White,No,No,No,"Yes, very religious",Pro-Choice,No,No,Behave no differently,5,1,4,Yes
3,8,Male,55,Independent,Moderate,High school or less,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-life,Yes,Yes,Less Willing,4,5,4,Yes
4,9,Male,64,Republican,Conservative,High school or less,White,Yes,Yes,Yes,No,Pro-life,No,No,Behave no differently,5,1,1,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,325,Male,21,Republican,Conservative,Some college,White,Yes,No,Yes,No,Pro-Choice,Yes,No,Less Willing,5,2,5,No
165,328,Female,41,Republican,Liberal,Graduate degree,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-Choice,Yes,No,Less Willing,5,2,2,No
166,329,Male,60,Republican,Conservative,Some college,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-life,No,Yes,Behave no differently,5,5,4,Yes
167,332,Female,51,Republican,Conservative,Graduate degree,White,Yes,Yes,Yes,"Yes, very religious",Pro-life,Yes,No,Less Willing,2,5,1,No


In [72]:
test_data

,id_num,Q1,Q2,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18
0,2,Female,78,Conservative,College degree,White,Yes,Yes,No,"Yes, very religious",Pro-Choice,Yes,Yes,Behave no differently,4,5,1,Yes
1,3,Male,59,Moderate,High school or less,Black,Yes,Yes,Yes,"Yes, very religious",Pro-Choice,No,No,More Willing,5,4,5,No
2,4,Male,59,Moderate,High school or less,White,Yes,No,Yes,"Yes, very religious",Pro-life,Yes,No,Behave no differently,4,5,1,Yes
3,6,Male,52,Moderate,Graduate degree,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-Choice,No,Yes,Less Willing,5,4,4,No
4,11,Female,33,Moderate,High school or less,White,No,No,Yes,"Yes, somewhat religious",Pro-Choice,No,No,More Willing,5,5,4,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,327,Female,68,Moderate,Graduate degree,White,Yes,No,No,"Yes, very religious",Pro-life,Yes,No,Behave no differently,5,5,2,No
162,330,Male,20,Moderate,High school or less,White,Yes,Yes,Yes,No,Pro-Choice,No,No,Less Willing,5,2,5,No
163,331,Male,65,Conservative,College degree,Latino,Yes,No,No,No,Pro-Choice,Yes,No,Behave no differently,5,2,1,No
164,333,Female,54,Moderate,Graduate degree,White,Yes,No,No,No,Pro-Choice,No,No,Behave no differently,5,1,5,Yes


Method 1

In [73]:
# Re-using the LabelEncoder for target variable
le_target = LabelEncoder()
y_train_encoded = le_target.fit_transform(train_data['political_affiliation'])

# Using LabelEncoder for encoding categorical features in the training and test datasets
train_data_encoded = train_data.apply(le.fit_transform)
test_data_encoded = test_data.apply(le.fit_transform)

# Splitting the training data into features and target
X_train = train_data_encoded.drop('political_affiliation', axis=1)

# Model selection and training with LDA
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train_encoded)

LinearDiscriminantAnalysis()

In [74]:
# Making predictions on the test dataset
lda_predictions = lda_model.predict(test_data_encoded)

# Creating a DataFrame for the submission with corrected label encoding
lda_submission = pd.DataFrame({
    "id_num": test_data['id_num'],
    "political_affiliation_predicted": le_target.inverse_transform(lda_predictions)
})

In [75]:
lda_submission

,id_num,political_affiliation_predicted
0,2,Republican
1,3,Democrat
2,4,Independent
3,6,Independent
4,11,Independent
...,...,...
161,327,Democrat
162,330,Independent
163,331,Democrat
164,333,Democrat


Method 2

In [76]:
# Encode categorical variables
le = LabelEncoder()
train_data_encoded = train_data.apply(le.fit_transform)
test_data_encoded = test_data.apply(le.fit_transform)
y_train_encoded = le.fit_transform(train_data['political_affiliation'])

X_train = train_data_encoded.drop('political_affiliation', axis=1)
# Model selection and training with Logistic Regression (OvR)
log_reg_ovr = LogisticRegression(multi_class='ovr', random_state=42)
log_reg_ovr.fit(X_train, y_train_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr', random_state=42)

In [77]:
# Making predictions on the test dataset
log_reg_predictions = log_reg_ovr.predict(test_data_encoded)

# Creating a DataFrame for the submission
log_reg_submission = pd.DataFrame({
    "id_num": test_data['id_num'],
    "political_affiliation_predicted": le.inverse_transform(log_reg_predictions)
})

In [78]:
log_reg_submission

,id_num,political_affiliation_predicted
0,2,Republican
1,3,Democrat
2,4,Independent
3,6,Independent
4,11,Independent
...,...,...
161,327,Democrat
162,330,Independent
163,331,Democrat
164,333,Democrat


In [79]:
# Encode your data
le = LabelEncoder()
train_data_encoded = train_data.apply(le.fit_transform)
test_data_encoded = test_data.apply(le.fit_transform)
y_train_encoded = le.fit_transform(train_data['political_affiliation'])

X_train = train_data_encoded.drop('political_affiliation', axis=1)

# Train the Logistic Regression model with One-vs-Rest
log_reg_ovr = LogisticRegression(multi_class='ovr', random_state=42)
log_reg_ovr.fit(X_train, y_train_encoded)

# Make predictions on the test dataset
log_reg_predictions = log_reg_ovr.predict(test_data_encoded)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [80]:
# Create a DataFrame for the submission
log_reg_submission_2 = pd.DataFrame({
    "id_num": test_data['id_num'],
    "political_affiliation_predicted": le.inverse_transform(log_reg_predictions)
})

# Convert the DataFrame to a CSV file
log_reg_submission_2.to_csv('log_reg_political_affiliation_predictions.csv', index=False)

In [81]:
log_reg_submission_2

,id_num,political_affiliation_predicted
0,2,Republican
1,3,Democrat
2,4,Independent
3,6,Independent
4,11,Independent
...,...,...
161,327,Democrat
162,330,Independent
163,331,Democrat
164,333,Democrat


Model 3

In [92]:
id_nums = test_data['id_num']

# Drop irrelevant columns (e.g., ID) for model training
X_train = train_data.drop(['id_num', 'political_affiliation'], axis=1)
y_train = train_data['political_affiliation']

# Encode categorical variables
X_train = pd.get_dummies(X_train)
test_data = pd.get_dummies(test_data)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the RandomForestClassifier
model = RandomForestClassifier(random_state=42)

# Create GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
# Find the best model to use for predictions
best_model = grid_search.best_estimator_
test_predictions_tuned = best_model.predict(X_val)

# Evaluate the tuned model
accuracy_tuned = accuracy_score(y_val, test_predictions_tuned)
print(f"Tuned Model Validation Accuracy: {accuracy_tuned}")

# Drop irrelevant columns (e.g., ID) for model prediction
X_test = test_data.drop(['id_num'], axis=1, errors='ignore')

## Make predictions on the test set using the tuned model
test_predictions_tuned = best_model.predict(X_test)

# Create a submission file with 'id_num' and tuned predictions
submission_tuned = pd.DataFrame({"id_num": id_nums, "political_affiliation_predicted": test_predictions_tuned})
submission_tuned = submission_tuned.set_index('id_num')  # Set 'id_num' as the index
submission_tuned.to_csv("submission_tuned.csv")

Tuned Model Validation Accuracy: 0.5588235294117647
